<p><font size="6"><b> Case study: air quality data of European monitoring stations (AirBase)</b></font></p><br>
**AirBase (The European Air quality dataBase): hourly measurements of all air quality monitoring stations from Europe. **


---

AirBase es la base de datos europea de calidad del aire mantenida por la Agencia Europea de Medio Ambiente (AEMA). Contiene datos e información de control de la calidad del aire enviada por los países participantes de Europa. La base de datos de la calidad del aire consta de una serie de tiempo de varios años de datos y estadísticas de medición de la calidad del aire para una serie de contaminantes del aire.


In [1]:
from IPython.display import HTML
HTML('<iframe src=http://www.eea.europa.eu/data-and-maps/data/airbase-the-european-air-quality-database-8#tab-data-by-country width=900 height=350></iframe>')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Algunos de los archivos de datos que están disponibles en AirBase se incluyeron en la carpeta de datos: las concentraciones horarias **de dióxido de nitrógeno (NO2)** para 4 estaciones de medición diferentes:


- FR04037 (PARIS 13eme): urban background site at Square de Choisy
- FR04012 (Paris, Place Victor Basch): urban traffic site at Rue d'Alesia
- BETR802: urban traffic site in Antwerp, Belgium
- BETN029: rural background site in Houtem, Belgium

See http://www.eea.europa.eu/themes/air/interactive/no2

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.display.max_rows = 8

# Processing a single file

Comenzaremos procesando uno de los archivos descargados (`BETR8010000800100hour.1-1-1990.31-12-2012`). Al mirar los datos, verá que no parece un buen archivo csv:

In [3]:
with open("data/BETR8010000800100hour.1-1-1990.31-12-2012") as f:
    print(f.readline())

1990-01-01	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0	-999.000	0



Así que tendremos que hacer un procesamiento manual.

Solo leyendo los datos delimitados por tabulaciones:

In [16]:
data = pd.read_csv("data/BETR8010000800100hour.1-1-1990.31-12-2012", sep='\t')#, header=None)

In [18]:
# data.head()
data

,1990-01-01,-999.000,0,-999.000.1,0.1,-999.000.2,0.2,-999.000.3,0.3,-999.000.4,...,-999.000.19,0.19,-999.000.20,0.20,-999.000.21,0.21,-999.000.22,0.22,-999.000.23,0.23
0,1990-01-02,-999.0,0,-999.0,0,-999.0,0,-999.0,0,-999.0,...,57.0,1,58.0,1,54.0,1,49.0,1,48.0,1
1,1990-01-03,51.0,1,50.0,1,47.0,1,48.0,1,51.0,...,84.0,1,75.0,1,-999.0,0,-999.0,0,-999.0,0
2,1990-01-04,-999.0,0,-999.0,0,-999.0,0,-999.0,0,-999.0,...,69.0,1,65.0,1,64.0,1,60.0,1,59.0,1
3,1990-01-05,51.0,1,51.0,1,48.0,1,50.0,1,51.0,...,-999.0,0,-999.0,0,-999.0,0,-999.0,0,-999.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8387,2012-12-28,26.5,1,28.5,1,35.5,1,32.0,1,35.5,...,46.5,1,42.5,1,38.5,1,30.5,1,26.5,1
8388,2012-12-29,21.5,1,16.5,1,13.0,1,13.0,1,16.0,...,21.0,1,22.0,1,20.5,1,20.0,1,15.0,1
8389,2012-12-30,11.5,1,9.5,1,7.5,1,7.5,1,10.0,...,25.0,1,18.5,1,17.0,1,15.5,1,12.5,1
8390,2012-12-31,9.5,1,8.5,1,8.5,1,8.5,1,10.5,...,21.0,1,16.5,1,14.5,1,16.5,1,15.0,1


¡Los datos anteriores claramente no están listos para ser utilizados! Cada fila contiene las 24 mediciones para cada hora del día y también contiene una bandera (0/1) que indica la calidad de los datos. Además, no hay una fila de encabezado con nombres de columna.

<div class="alert alert-success">

<b>EXERCISE</b>: <br><br> Limpia este marco de datos usando más opciones de `read_csv` (see its [docstring](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html))

 <ul>
  <li> especifique el delimitador correcto </li>
  <li> especificar que los valores de -999 y -9999 deben considerarse como NaN </li>
  <li> especifique sus propios nombres de columna (para ver cómo se componen los nombres de columna, consulte http://stackoverflow.com/questions/6356041/python-intertwining-two-lists)
</ul>
</div>

In [10]:
# Column names: list consisting of 'date' and then intertwined the hour of the day and 'flag'
hours = ["{:02d}".format(i) for i in range(24)]
column_names = ['date'] + [item for pair in zip(hours, ['flag']*24) for item in pair]

In [11]:
data = pd.read_csv("data/BETR8010000800100hour.1-1-1990.31-12-2012",
                   sep='\t', header=None, names=column_names, na_values=[-999, -9999])

ValueError: Duplicate names are not allowed.

In [12]:
data.head()

,1990-01-01,-999.000,0,-999.000.1,0.1,-999.000.2,0.2,-999.000.3,0.3,-999.000.4,...,-999.000.19,0.19,-999.000.20,0.20,-999.000.21,0.21,-999.000.22,0.22,-999.000.23,0.23
0,1990-01-02,-999.0,0,-999.0,0,-999.0,0,-999.0,0,-999.0,...,57.0,1,58.0,1,54.0,1,49.0,1,48.0,1
1,1990-01-03,51.0,1,50.0,1,47.0,1,48.0,1,51.0,...,84.0,1,75.0,1,-999.0,0,-999.0,0,-999.0,0
2,1990-01-04,-999.0,0,-999.0,0,-999.0,0,-999.0,0,-999.0,...,69.0,1,65.0,1,64.0,1,60.0,1,59.0,1
3,1990-01-05,51.0,1,51.0,1,48.0,1,50.0,1,51.0,...,-999.0,0,-999.0,0,-999.0,0,-999.0,0,-999.0,0
4,1990-01-06,-999.0,0,-999.0,0,-999.0,0,-999.0,0,-999.0,...,-999.0,0,-999.0,0,-999.0,0,-999.0,0,-999.0,0


Por el bien de este tutorial, ignoraremos las columnas 'bandera' (que indican la calidad de los datos).

<div class="alert alert-success">

<b>EXERCISE</b>:
<br><br>
Eliminar todas las columnas de 'bandera' ('bandera1', 'bandera2', ...)

In [13]:
flag_columns = [col for col in data.columns if 'flag' in col]
# we can now use this list to drop these columns

In [14]:
data = data.drop(flag_columns, axis=1)

In [15]:
data.head()

,1990-01-01,-999.000,0,-999.000.1,0.1,-999.000.2,0.2,-999.000.3,0.3,-999.000.4,...,-999.000.19,0.19,-999.000.20,0.20,-999.000.21,0.21,-999.000.22,0.22,-999.000.23,0.23
0,1990-01-02,-999.0,0,-999.0,0,-999.0,0,-999.0,0,-999.0,...,57.0,1,58.0,1,54.0,1,49.0,1,48.0,1
1,1990-01-03,51.0,1,50.0,1,47.0,1,48.0,1,51.0,...,84.0,1,75.0,1,-999.0,0,-999.0,0,-999.0,0
2,1990-01-04,-999.0,0,-999.0,0,-999.0,0,-999.0,0,-999.0,...,69.0,1,65.0,1,64.0,1,60.0,1,59.0,1
3,1990-01-05,51.0,1,51.0,1,48.0,1,50.0,1,51.0,...,-999.0,0,-999.0,0,-999.0,0,-999.0,0,-999.0,0
4,1990-01-06,-999.0,0,-999.0,0,-999.0,0,-999.0,0,-999.0,...,-999.0,0,-999.0,0,-999.0,0,-999.0,0,-999.0,0


Ahora, queremos darle una nueva forma: nuestro objetivo es tener las diferentes horas como índices de fila, fusionados con la fecha en un índice de fecha y hora. Aquí tenemos un marco de datos amplio y largo, y queremos que sea una serie de tiempo larga y estrecha.

<div class="alert alert-info">

<b>REMEMBER</b>:

 <ul>
  <li> Resumen: remodelar sus datos con [`stack` y` unstack`] (./ pandas_07_reshaping_data.ipynb) </li>
</ul>


<img src="img/schema-stack.svg" width=70%>

</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

<br><br>

Cambie la forma del marco de datos a una serie temporal.
El resultado final debería verse así: <br><br>


<div class='center'>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>BETR801</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1990-01-02 09:00:00</th>
      <td>48.0</td>
    </tr>
    <tr>
      <th>1990-01-02 12:00:00</th>
      <td>48.0</td>
    </tr>
    <tr>
      <th>1990-01-02 13:00:00</th>
      <td>50.0</td>
    </tr>
    <tr>
      <th>1990-01-02 14:00:00</th>
      <td>55.0</td>
    </tr>
    <tr>
      <th>...</th>
      <td>...</td>
    </tr>
    <tr>
      <th>2012-12-31 20:00:00</th>
      <td>16.5</td>
    </tr>
    <tr>
      <th>2012-12-31 21:00:00</th>
      <td>14.5</td>
    </tr>
    <tr>
      <th>2012-12-31 22:00:00</th>
      <td>16.5</td>
    </tr>
    <tr>
      <th>2012-12-31 23:00:00</th>
      <td>15.0</td>
    </tr>
  </tbody>
</table>
<p style="text-align:center">170794 rows × 1 columns</p>
</div>

 <ul>
  <li> Cambie la forma del marco de datos para que cada fila conste de una observación para una combinación de fecha y hora </li>
  <li> Cuando tenga los valores de fecha y hora como dos columnas, combine estas columnas en una fecha y hora (consejo: las columnas de cadena se pueden sumar para concatenar las cadenas) y elimine las columnas originales </li>
  <li> Establezca los nuevos valores de fecha y hora como índice y elimine las columnas originales con los valores de fecha y hora </li>
</ul>


**NOTE**: This is an advanced exercise. Do not spend too much time on it and don't hesitate to look at the solutions.

</div>



In [ ]:
# we use stack to reshape the data to move the hours (the column labels) into a column.
# But we don't want to move the 'date' column label, therefore we first set this as the index.
# You can check the difference with "data.stack()"
data2 = data.set_index('date')
data_stacked = data2.stack()
data_stacked.head()

In [ ]:
# We reset the index to have the date and hours available as columns
data_stacked = data_stacked.reset_index()
data_stacked.head()

In [ ]:
# Now we combine the dates and the hours into a datetime, and set this as the index
data_stacked.index = pd.to_datetime(data_stacked['date'] + data_stacked['level_1'], format="%Y-%m-%d%H")

In [ ]:
# Drop the origal date and hour columns
data_stacked = data_stacked.drop(['date', 'level_1'], axis=1)
data_stacked.head()

In [ ]:
# rename the remaining column to the name of the measurement station
data_stacked = data_stacked.rename(columns={0: 'BETR801'})

In [ ]:
data_stacked.head()

Our final data is now a time series. In pandas, this means that the index is a `DatetimeIndex`:

In [ ]:
data_stacked.index

In [ ]:
data_stacked.plot()

# Processing a collection of files

Ahora hemos visto los pasos del código para procesar uno de los archivos. Sin embargo, tenemos varios archivos para las diferentes estaciones con la misma estructura. Por lo tanto, para no tener que repetir el código real, hagamos una función a partir de los pasos que hemos visto anteriormente.

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
   <li> Escriba una función `read_airbase_file (nombre de archivo, estación)`, utilizando los pasos anteriores para leer y procesar los datos, y que devuelva una serie de tiempo procesada. </li>
</ul>
</div>

In [ ]:
def read_airbase_file(filename, station):
    """
    Read hourly AirBase data files.

    Parameters
    ----------
    filename : string
        Path to the data file.
    station : string
        Name of the station.

    Returns
    -------
    DataFrame
        Processed dataframe.
    """

    ...

    return ...

In [ ]:
def read_airbase_file(filename, station):
    """
    Read hourly AirBase data files.

    Parameters
    ----------
    filename : string
        Path to the data file.
    station : string
        Name of the station.

    Returns
    -------
    DataFrame
        Processed dataframe.
    """

    # construct the column names
    hours = ["{:02d}".format(i) for i in range(24)]
    colnames = ['date'] + [item for pair in zip(hours, ['flag']*24) for item in pair]

    # read the actual data
    data = pd.read_csv(filename, sep='\t', header=None, na_values=[-999, -9999], names=colnames)

    # drop the 'flag' columns
    data = data.drop([col for col in data.columns if 'flag' in col], axis=1)

    # reshape
    data = data.set_index('date')
    data_stacked = data.stack()
    data_stacked = data_stacked.reset_index()

    # parse to datetime and remove redundant columns
    data_stacked.index = pd.to_datetime(data_stacked['date'] + data_stacked['level_1'], format="%Y-%m-%d%H")
    data_stacked = data_stacked.drop(['date', 'level_1'], axis=1)
    data_stacked = data_stacked.rename(columns={0: station})

    return data_stacked

Test the function on the data file from above:

In [ ]:
filename = "data/BETR8010000800100hour.1-1-1990.31-12-2012"
station = filename.split("/")[-1][:7]

In [ ]:
station

In [ ]:
test = read_airbase_file(filename, station)
test.head()

Ahora queremos usar esta función para leer todos los diferentes archivos de datos de AirBase y combinarlos en un solo marco de datos.

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li> Utilice la función `glob.glob` para enumerar los 4 archivos de datos de AirBase que están incluidos en el directorio 'data' y llame al resultado` data_files`. </li>
</ul>
</div>

In [ ]:
import glob

In [ ]:
data_files = glob.glob("data/*0008001*")
data_files

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li> Recorra los archivos de datos, lea y procese el archivo usando nuestra función definida y agregue el marco de datos a una lista. </li>
  <li> Combine los diferentes DataFrames en la lista en un solo DataFrame donde las diferentes columnas son las diferentes estaciones. Llame al resultado "datos_combinados". </li>

</ul>
</div>

In [ ]:
dfs = []

for filename in data_files:
    station = filename.split("/")[-1][:7]
    df = read_airbase_file(filename, station)
    dfs.append(df)

In [ ]:
combined_data = pd.concat(dfs, axis=1)

In [ ]:
combined_data.head()

Finalmente, no queremos tener que repetir esto cada vez que usemos los datos. Por lo tanto, guardemos los datos procesados en un archivo csv.

In [ ]:
combined_data.to_csv("airbase_data.csv")

# Working with time series data

Procesamos los archivos de datos individuales anteriores y los guardamos en un archivo csv `airbase_data.csv`. Vamos a importar el archivo aquí (si no completó los ejercicios anteriores, una versión del conjunto de datos también está disponible en `data / airbase_data.csv`):

In [ ]:
alldata = pd.read_csv('airbase_data.csv', index_col=0, parse_dates=True)

Solo usamos los datos de 1999 en adelante:

In [ ]:
data = alldata['1999':].copy()

Una primera exploración con las funciones **típicas**:

In [ ]:
data.head() # tail()

In [ ]:
data.info()

In [ ]:
data.describe(percentiles=[0.1, 0.5, 0.9])

Visualización rápida de los datos

In [ ]:
data.plot(kind='box', ylim=[0,250])

In [ ]:
data['BETR801'].plot(kind='hist', bins=50)

In [ ]:
data.plot(figsize=(12,6))

Esto no dice demasiado ..

Podemos seleccionar parte de los datos (por ejemplo, los últimos 500 puntos de datos):

In [ ]:
data[-500:].plot(figsize=(12,6))

## Exercises

<div class="alert alert-warning">

<b>REMINDER</b>: <br><br>

Eche un vistazo al [Cuaderno de la serie temporal] (05 - Datos de la serie temporal.ipynb) cuando necesite más información sobre ...
    
 <ul>
  <li>`resample`</li>
  <li>indexación de cadenas de DateTimeIndex</li>
</ul><br><br>

</div>

<div class="alert alert-success">
    <b>QUESTION</b>: trazar la concentración mensual media y mediana de la estación 'FR04037' para los años 2009-2012
</div>

In [ ]:
data.loc['2009':, 'FR04037'].resample('M').mean().plot()
data.loc['2009':, 'FR04037'].resample('M').median().plot()

In [ ]:
data.loc['2009':, 'FR04037'].resample('M').agg(['mean', 'median']).plot()

<div class="alert alert-success">
    <b>QUESTION</b>: trazar la concentración diaria mínima y máxima mensual de la estación 'BETR801'
</div>

In [ ]:
daily = data['FR04037'].resample('D').mean()

In [ ]:
daily.resample('M').agg(['min', 'max']).plot()

<div class="alert alert-success">
    <b>QUESTION</b>: hacer un diagrama de barras de la media de las estaciones en el año de 2012
</div>

In [ ]:
data['2012'].mean().plot(kind='bar')

<div class="alert alert-success">
    <b>QUESTION</b>: ¿La evolución de los promedios anuales con y la media general de todas las estaciones (indicar la media general con una línea negra más gruesa)?
</div>

In [ ]:
data.resample('A').mean().plot()
data.mean(axis=1).resample('A').mean().plot(color='k', linestyle='--', linewidth=4)

### Combination with groupby

`resample` en realidad puede verse como un tipo específico de` groupby`. P.ej. tomar medias anuales con `data.resample ('A', 'mean')` es equivalente a `data.groupby (data.index.year) .mean ()` (solo el resultado de `resample` todavía tiene un DatetimeIndex) .

In [ ]:
data.groupby(data.index.year).mean().plot()

Pero, `groupby` es más flexible y también puede hacer remuestreos que no dan como resultado una nueva serie de tiempo continua, p. Ej. agrupando por hora del día para obtener el ciclo diurno.

<div class="alert alert-success">
    <b>QUESTION</b>: ¿Cómo se ve el * perfil mensual típico * para las diferentes estaciones?
</div>

1 \. agregue una columna al marco de datos que indique el mes (valor entero de 1 a 12):

In [ ]:
data['month'] = data.index.month

2\. Now, we can calculate the mean of each month over the different years:

In [ ]:
data.groupby('month').mean()

3 \. trazar el perfil mensual típico de las diferentes estaciones:

In [ ]:
data.groupby('month').mean().plot()

In [ ]:
data = data.drop('month', axis=1, errors='ignore')

<div class="alert alert-success">
    <b>QUESTION</b>: trazar los percentiles semanales del 95% de la concentración en 'BETR801' y 'BETN029' para 2011
</div>

In [ ]:
df2011 = data['2011'].dropna()

In [ ]:
df2011 = data['2011'].dropna()
df2011.groupby(df2011.index.week)[['BETN029', 'BETR801']].quantile(0.95).plot()

In [ ]:
df2011[['BETN029', 'BETR801']].resample('W').agg(lambda x: x.quantile(0.75)).plot()

<div class="alert alert-success">
    <b>QUESTION</b>: ¿El perfil diurno típico de las diferentes estaciones?
</div>

In [ ]:
data.groupby(data.index.hour).mean().plot()

<div class="alert alert-success">
    <b>QUESTION</b>: ¿Cuál es el número de superaciones de los valores horarios por encima del límite europeo de 200 µg / m3 para cada año / estación?
</div>

In [ ]:
exceedances = data > 200

In [ ]:
# group by year and count exceedances (sum of boolean)
exceedances = exceedances.groupby(exceedances.index.year).sum()

In [ ]:
exceedances

In [ ]:
ax = exceedances.loc[2005:].plot(kind='bar')
ax.axhline(18, color='k', linestyle='--')

<div class="alert alert-success">
    <b>QUESTION</b>: ¿Se ha superado el valor límite anual de 40 µg / m3 desde 2000?
</div>

In [ ]:
yearly = data['2000':].resample('A').mean()

In [ ]:
(yearly > 40).sum()

In [ ]:
yearly.plot()
plt.axhline(40, linestyle='--', color='k')

<div class="alert alert-success">
    <b>QUESTION</b>: ¿Cuál es la diferencia en el perfil diurno típico entre los días de la semana y los fines de semana? (y visualízalo)
</div>

In [ ]:
data.index.weekday?

In [ ]:
data['weekday'] = data.index.weekday

Add a column indicating week/weekend

In [ ]:
data['weekend'] = data['weekday'].isin([5, 6])

In [ ]:
data_weekend = data.groupby(['weekend', data.index.hour]).mean()
data_weekend.head()

In [ ]:
data_weekend_BETR801 = data_weekend['BETR801'].unstack(level=0)
data_weekend_BETR801.head()

In [ ]:
data_weekend_BETR801.plot()

In [ ]:
data = data.drop(['weekday', 'weekend'], axis=1)

<div class="alert alert-success">
    <b>QUESTION</b>: Visualice el perfil de semana típico para las diferentes estaciones como diagramas de caja (donde los valores en un diagrama de caja son los promedios diarios para las diferentes semanas para un determinado día de la semana).
</div>

Consejo: el método de diagrama de caja de un DataFrame espera los datos para los diferentes cuadros en diferentes columnas). Para esto, puedes usar `pivot_table` como una combinación de` groupby` y `unstack`

In [ ]:
# add a weekday and week column
data['weekday'] = data.index.weekday
data['week'] = data.index.week
data.head()

In [ ]:
# pivot table so that the weekdays are the different columns
data_pivoted = data['2012'].pivot_table(columns='weekday', index='week', values='BETR801')
data_pivoted.head()

In [ ]:
box = data_pivoted.boxplot()

An alternative method using `groupby` and `unstack`:

In [ ]:
data['2012'].groupby(['weekday', 'week'])['BETR801'].mean().unstack(level=0).boxplot();

<div class="alert alert-success">
    <b>QUESTION</b>: La media máxima diaria de 8 horas debe ser inferior a 100 µg / m³. ¿Cuál es el número de superaciones de este límite para cada año / estación?
</div>

Consejo: eche un vistazo al método `rolling` para realizar operaciones de ventana móvil.

Nota: este no es un límite real para el NO2, pero es un buen ejercicio para introducir el método de `rolling`. Otros contaminantes, como el 03, tienen realmente ese tipo de valores límite.

In [ ]:
exceedances = data.rolling(8).mean().resample('D').max() > 100
exceedances = exceedances.groupby(exceedances.index.year).sum()
ax = exceedances.loc[2005:].plot(kind='bar')

<div class="alert alert-success">
    <b>QUESTION</b>: Calcula la correlación entre las diferentes estaciones.
</div>


In [ ]:
data[['BETR801', 'BETN029', 'FR04037', 'FR04012']].corr()

In [ ]:
data[['BETR801', 'BETN029', 'FR04037', 'FR04012']].resample('D').mean().corr()